In [1]:
import pandas as pd
import glob
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from joblib import dump

import keras
from keras.engine import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, concatenate
from keras.callbacks import ModelCheckpoint
from tensorflow import set_random_seed
set_random_seed(12)

Using TensorFlow backend.
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaco

# Preprocessing

In [2]:
# Sequence for the LSTM network
backlook = 4

# Size of data split for testing
train_size = 0.9

In [3]:
path = './datasets/fundamental_data'
all_files = glob.glob(path + "/*.csv")

time_li = []
nexus_li = []
y_vals = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0).sort_values(by='fiscalDateEnding')
    
    if len(df) > 4:
        df['growth'] = ((df['nextClossingVal']/df['clossingVal'])*100)-100
        df = df.drop(columns=['Unnamed: 0', 'fiscalDateEnding', 'reportedCurrency_x', 'reportedCurrency_y', 'reportedCurrency', 'nextClossingVal', 'clossingVal'])
        df = df.replace('None', 0).fillna(0)
        df = df[df['growth'] != 0]
        
        for i in range(len(df)-backlook):
            time_li.append(df.iloc[i:i+backlook])
            y_vals.append(df.iloc[[i+backlook]]['growth'].values[0])
            nexus_li.append(df.drop(columns='growth').iloc[[i+backlook]])

In [4]:
time_arr = np.zeros((len(time_li), 4, 99))
nexus_arr = np.zeros((len(time_li), 98))
y_arr = np.array(y_vals).reshape(-1, 1)

for i in range(len(time_li)):
    time_arr[i] = time_li[i].to_numpy()
    nexus_arr[i] = nexus_li[i].to_numpy().reshape(98,)

In [5]:
# Train-Test split
split = int(len(time_arr) * train_size)

train_time = time_arr[:split]
train_nexus = nexus_arr[:split]
train_y = y_arr[:split]

test_time = time_arr[split:]
test_nexus = nexus_arr[split:]
test_y = y_arr[split:]

In [6]:
# Normalise series
time_normaliser = preprocessing.MinMaxScaler()
train_time = time_normaliser.fit_transform(train_time.reshape(train_time.shape[0]*train_time.shape[1], train_time.shape[2])).reshape(train_time.shape)
test_time = time_normaliser.transform(test_time.reshape(test_time.shape[0]*test_time.shape[1], test_time.shape[2])).reshape(test_time.shape)

# Normalise nexus
nexus_normaliser = preprocessing.MinMaxScaler()
train_nexus = nexus_normaliser.fit_transform(train_nexus)
test_nexus = nexus_normaliser.transform(test_nexus)

# Y normaliser
y_normaliser = preprocessing.MinMaxScaler()
train_y = y_normaliser.fit_transform(train_y)
test_y = y_normaliser.transform(test_y)

In [7]:
# Save scalers for future use
dump(time_normaliser, './normalisers/fundamental_boosted_x-time_normaliser.joblib')
dump(nexus_normaliser, './normalisers/fundamental_boosted_x-nexus_normaliser.joblib')
dump(y_normaliser, './normalisers/fundamental_boosted_y_normaliser.joblib')

['./normalisers/fundamental_boosted_y_normaliser.joblib']

# Model selection

##### Model constructor

In [8]:
def build_complex(x_time, x_nexus, y, batch_size=512, epochs=24, **params):
    
    # List of parameters
    if 'lstmsize' not in params: params['lstmsize'] = x_time.shape[1]
    if 'density' not in params: params['density'] = x_nexus.shape[1]
    if 'merge_density' not in params: params['merge_density'] = int((params['lstmsize']//1.5)*2) + x_nexus.shape[1]
    if 'activation' not in params: params['activation'] = 'relu'
    if 'twice' not in params: params['twice'] = False
    if 'optimizer' not in params: params['optimizer'] = 'adam'
    if 'shuffle' not in params: params['shuffle'] = False
        
    # Time model definition
    model_time = Sequential()
    
    model_time.add(LSTM(params['lstmsize'], input_shape=x_time.shape[1:], return_sequences=params['twice']))
    
    if 'dropout' in params:
        model_time.add(Dropout(params['dropout']))
    
    if params['twice']:
        model_time.add(LSTM(params['lstmsize']))
        
        if 'dropout' in params:
            model_time.add(Dropout(params['dropout']))
    
    # Nexus model definition
    model_nexus = Sequential()
    
    model_nexus.add(Dense(params['density'], input_shape=(98,), activation=params['activation']))
    
    # Ending pipe
    model = concatenate([model_time.output, model_nexus.output])
    
    model = Dense(params['merge_density'], activation=params['activation'])(model)
    
    if 'full_density' in params and params['full_density']:
        density = params['merge_density']//2
        while density >= 12:
            model = Dense(density, activation=params['activation'])(model)
            density //= 2
            
    model = Dense(1, activation='linear')(model)
    
    model = Model([model_time.input, model_nexus.input], model)
    
    model.compile(loss='mse', optimizer=params['optimizer'])
    
    if 'callbacks' in params:
        callback = model.fit(x=[x_time, x_nexus], y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'],
                            callbacks=params['callbacks'])
    else:
        callback = model.fit(x=[x_time, x_nexus], y=y, validation_split=0.1, batch_size=batch_size, epochs=epochs, shuffle=params['shuffle'])
    
    return [model, callback, params]

##### Evolutive algorith to search for the most optimal model

In [9]:
def set_genes(x_time, x_nexus, y, population_size, population=[]):
    if population_size-len(population) < 0:
        print('population size must be greater or equal than actual population')
        return
    
    if len(population) > 0:
        if len(population) < 3:
            print('population should be empty or 3 at least')
            return
        
        population[2] = breed_genes(x_time, x_nexus, y, population[0][2].copy())
        population[1] = breed_genes(x_time, x_nexus, y, combine=(population[0][2], population[1][2]))
        population = population[:3]
    
    for _ in range(population_size-len(population)):
        subject = breed_genes(x_time, x_nexus, y)
        population.append(subject)
        
    return population

def breed_genes(x_time, x_nexus, y, genes={}, combine=None):
    genes['x_time'] = x_time
    genes['x_nexus'] = x_nexus
    genes['y'] = y
    
    if type(combine) is list or type(combine) is tuple:
        genes['lstmsize'] = combine[np.random.randint(0,2)]['lstmsize']
        genes['density'] = combine[np.random.randint(0,2)]['density']
        genes['merge_density'] = combine[np.random.randint(0,2)]['merge_density']
        genes['activation'] = combine[np.random.randint(0,2)]['activation']
        genes['twice'] = combine[np.random.randint(0,2)]['twice']
        genes['optimizer'] = combine[np.random.randint(0,2)]['optimizer']
        genes['shuffle'] = combine[np.random.randint(0,2)]['shuffle']
        
        genes['dropout'] = combine[np.random.randint(0,2)].get('dropout')
        if genes['dropout'] is None: del genes['dropout']
        genes['full_density'] = combine[np.random.randint(0,2)].get('full_density')
        if genes['full_density'] is None: del genes['full_density']
            
    else:
        if np.random.randint(0,3) == 1:
            genes['lstmsize'] = int((np.random.randint(x_time.shape[1],x_time.shape[1]*2)//2)*2)

        if np.random.randint(0,3) == 1:
            genes['density'] = int((np.random.randint(x_nexus.shape[1]//2,x_nexus.shape[1]*2.66)//2)*2)

        if np.random.randint(0,3) == 1:
            genes['merge_density'] = int((np.random.randint(x_time.shape[1]+(x_nexus.shape[1]//2),
                                                            (x_time.shape[1]*2)+(x_nexus.shape[1]*2.66))//2)*2)

        if np.random.randint(0,3) == 1:
            activation = [
                'relu',
                'sigmoid',
                'softplus',
                'softsign',
                'tanh',
                'selu',
                'elu',
                'exponential'
            ]
            genes['activation'] = activation[np.random.randint(0,8)]

        if np.random.randint(0,3) == 1:
            genes['twice'] = True

        if np.random.randint(0,3) == 1:
            optimizer = [
                'sgd',
                'rmsprop',
                'adam',
                'adadelta',
                'adagrad',
                'adamax',
                'nadam'
            ]
            genes['optimizer'] = optimizer[np.random.randint(0,7)]

        if np.random.randint(0,3) == 1:
            genes['shuffle'] = True

        if np.random.randint(0,3) == 1:
            genes['dropout'] = np.random.randint(1,4)/10

        if np.random.randint(0,3) == 1:
            genes['full_density'] = True
            
    new_model = build_complex(**genes)
        
    return new_model

In [10]:
def breed_population(x_time, x_nexus, y, generations, population_size, population=[]):
    
    if generations > 1 and population_size < 3:
        print('population size should be of a minimum of 3 for more than one generation')
        return
    
    for g in range(generations):
        print(f'\nGENERATION {g}\n')
        population = set_genes(x_time, x_nexus, y, population_size, population)
        population = sorted(population, key=lambda x: x[1].history['val_loss'][-1]+x[1].history['loss'][-1])
        
    return population

In [11]:
best_models = breed_population(train_time, train_nexus, train_y, generations=20, population_size=10)


GENERATION 0

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 18021 samples, validate on 2003 samples
Epoch 1/24
18021/18021 [==============================] - 1s 38us/step - loss: 0.0648 - val_loss: 1.1179e-04
Epoch 2/24
18021/18021 [==============================] - 0s 12us/step - loss: 1.5341e-04 - val_loss: 1.0018e-04
Epoch 3/24
18021/18021 [==============================] - 0s 12us/step - loss: 1.5282e-04 - val_loss: 9.5590e-05
Epoch 4/24
18021/18021 [==============================] - 0s 12us/step - loss: 1.4268e-04 - val_loss: 9.3689e-05
Epoch 5/24
18021/18021 [==============================] - 0s 12us/step - loss: 1.3990e-04 - val_loss: 9.1621e-05
Epoch 6/24
18021/18021 [==============================] - 0s 12us/step - loss: 1.3813e-04 - val_loss: 8.9899e-05
Epoch 7/24
18021/18021 [==============================] - 0s 12us/step - loss: 1.3691e-04 - val_loss: 8.8997e-05
Epoch 8/24
18021/18021 [=====

18021/18021 [==============================] - 0s 21us/step - loss: 1.2344e-04 - val_loss: 6.5906e-05
Epoch 19/24
18021/18021 [==============================] - 0s 21us/step - loss: 1.2345e-04 - val_loss: 6.5938e-05
Epoch 20/24
18021/18021 [==============================] - 0s 22us/step - loss: 1.2322e-04 - val_loss: 6.6047e-05
Epoch 21/24
18021/18021 [==============================] - 0s 21us/step - loss: 1.2320e-04 - val_loss: 6.7008e-05
Epoch 22/24
18021/18021 [==============================] - 0s 22us/step - loss: 1.2343e-04 - val_loss: 6.5795e-05
Epoch 23/24
18021/18021 [==============================] - 0s 19us/step - loss: 1.2334e-04 - val_loss: 6.5923e-05
Epoch 24/24
18021/18021 [==============================] - 0s 20us/step - loss: 1.2328e-04 - val_loss: 6.5898e-05
Train on 18021 samples, validate on 2003 samples
Epoch 1/24
18021/18021 [==============================] - 1s 62us/step - loss: nan - val_loss: nan
Epoch 2/24
18021/18021 [==============================] - 0s 20us/

Epoch 20/24
18021/18021 [==============================] - 0s 21us/step - loss: 1.2757e-04 - val_loss: 7.2210e-05
Epoch 21/24
18021/18021 [==============================] - 0s 21us/step - loss: 1.2740e-04 - val_loss: 7.2037e-05
Epoch 22/24
18021/18021 [==============================] - 0s 20us/step - loss: 1.2737e-04 - val_loss: 7.2483e-05
Epoch 23/24
18021/18021 [==============================] - 0s 20us/step - loss: 1.2749e-04 - val_loss: 1.0663e-04
Epoch 24/24
18021/18021 [==============================] - 0s 20us/step - loss: 1.2828e-04 - val_loss: 7.2856e-05
Train on 18021 samples, validate on 2003 samples
Epoch 1/24
18021/18021 [==============================] - 1s 76us/step - loss: 0.0159 - val_loss: 0.0012
Epoch 2/24
18021/18021 [==============================] - 0s 21us/step - loss: 5.8116e-04 - val_loss: 1.3629e-04
Epoch 3/24
18021/18021 [==============================] - 0s 20us/step - loss: 2.3814e-04 - val_loss: 1.2614e-04
Epoch 4/24
18021/18021 [==========================

18021/18021 [==============================] - 0s 21us/step - loss: 5.0037e-04 - val_loss: 3.6524e-04
Epoch 20/24
18021/18021 [==============================] - 0s 21us/step - loss: 4.2956e-04 - val_loss: 8.7736e-04
Epoch 21/24
18021/18021 [==============================] - 0s 21us/step - loss: 4.8800e-04 - val_loss: 2.3461e-04
Epoch 22/24
18021/18021 [==============================] - 0s 21us/step - loss: 4.6143e-04 - val_loss: 2.7534e-04
Epoch 23/24
18021/18021 [==============================] - 0s 21us/step - loss: 4.4598e-04 - val_loss: 5.1070e-04
Epoch 24/24
18021/18021 [==============================] - 0s 20us/step - loss: 4.2384e-04 - val_loss: 4.5755e-04
Train on 18021 samples, validate on 2003 samples
Epoch 1/24
18021/18021 [==============================] - 2s 84us/step - loss: 0.0464 - val_loss: 0.0066
Epoch 2/24
18021/18021 [==============================] - 0s 21us/step - loss: 0.0061 - val_loss: 0.0069
Epoch 3/24
18021/18021 [==============================] - 0s 21us/ste

18021/18021 [==============================] - 0s 21us/step - loss: 1.2407e-04 - val_loss: 6.5644e-05
Epoch 19/24
18021/18021 [==============================] - 0s 20us/step - loss: 1.2402e-04 - val_loss: 6.5782e-05
Epoch 20/24
18021/18021 [==============================] - 0s 21us/step - loss: 1.2350e-04 - val_loss: 6.7013e-05
Epoch 21/24
18021/18021 [==============================] - 0s 21us/step - loss: 1.2373e-04 - val_loss: 6.5622e-05
Epoch 22/24
18021/18021 [==============================] - 0s 22us/step - loss: 1.2364e-04 - val_loss: 6.5594e-05
Epoch 23/24
18021/18021 [==============================] - 0s 21us/step - loss: 1.2496e-04 - val_loss: 6.5552e-05
Epoch 24/24
18021/18021 [==============================] - 0s 21us/step - loss: 1.2343e-04 - val_loss: 6.6749e-05
Train on 18021 samples, validate on 2003 samples
Epoch 1/24
18021/18021 [==============================] - 2s 93us/step - loss: 0.0634 - val_loss: 6.6666e-05
Epoch 2/24
18021/18021 [==============================] 

...

Epoch 10/24
18021/18021 [==============================] - 1s 63us/step - loss: 1.2259e-04 - val_loss: 6.5429e-05
Epoch 11/24
18021/18021 [==============================] - 1s 61us/step - loss: 1.2281e-04 - val_loss: 6.5094e-05
Epoch 12/24
18021/18021 [==============================] - 1s 61us/step - loss: 1.2261e-04 - val_loss: 6.5317e-05
Epoch 13/24
18021/18021 [==============================] - 1s 61us/step - loss: 1.2265e-04 - val_loss: 6.5482e-05
Epoch 14/24
18021/18021 [==============================] - 1s 61us/step - loss: 1.2264e-04 - val_loss: 6.5815e-05
Epoch 15/24
18021/18021 [==============================] - 1s 61us/step - loss: 1.2266e-04 - val_loss: 6.5097e-05
Epoch 16/24
18021/18021 [==============================] - 1s 64us/step - loss: 1.2247e-04 - val_loss: 6.5746e-05
Epoch 17/24
18021/18021 [==============================] - 1s 61us/step - loss: 1.2264e-04 - val_loss: 6.5135e-05
Epoch 18/24
18021/18021 [==============================] - 1s 62us/step - loss: 1.2261e-

Epoch 11/24
18021/18021 [==============================] - 1s 65us/step - loss: 0.0015 - val_loss: 0.0041
Epoch 12/24
18021/18021 [==============================] - 1s 65us/step - loss: 0.0019 - val_loss: 8.6585e-05
Epoch 13/24
18021/18021 [==============================] - 1s 65us/step - loss: 0.0019 - val_loss: 0.0025
Epoch 14/24
18021/18021 [==============================] - 1s 68us/step - loss: 0.0012 - val_loss: 1.1892e-04
Epoch 15/24
18021/18021 [==============================] - 1s 69us/step - loss: 0.0014 - val_loss: 9.1849e-05
Epoch 16/24
18021/18021 [==============================] - 1s 66us/step - loss: 0.0010 - val_loss: 0.0016
Epoch 17/24
18021/18021 [==============================] - 1s 66us/step - loss: 0.0013 - val_loss: 1.2964e-04
Epoch 18/24
18021/18021 [==============================] - 1s 72us/step - loss: 7.4546e-04 - val_loss: 0.0015
Epoch 19/24
18021/18021 [==============================] - 1s 71us/step - loss: 0.0018 - val_loss: 1.2056e-04
Epoch 20/24
18021/1802

18021/18021 [==============================] - 1s 71us/step - loss: 1.2247e-04 - val_loss: 6.5428e-05
Epoch 14/24
18021/18021 [==============================] - 1s 70us/step - loss: 1.2247e-04 - val_loss: 6.5109e-05
Epoch 15/24
18021/18021 [==============================] - 1s 71us/step - loss: 1.2260e-04 - val_loss: 6.5377e-05
Epoch 16/24
18021/18021 [==============================] - 1s 71us/step - loss: 1.2242e-04 - val_loss: 6.5172e-05
Epoch 17/24
18021/18021 [==============================] - 1s 72us/step - loss: 1.2241e-04 - val_loss: 6.5290e-05
Epoch 18/24
18021/18021 [==============================] - 1s 72us/step - loss: 1.2241e-04 - val_loss: 6.5093e-05
Epoch 19/24
18021/18021 [==============================] - 1s 73us/step - loss: 1.2239e-04 - val_loss: 6.5111e-05
Epoch 20/24
18021/18021 [==============================] - 1s 70us/step - loss: 1.2241e-04 - val_loss: 6.5233e-05
Epoch 21/24
18021/18021 [==============================] - 1s 72us/step - loss: 1.2238e-04 - val_los

18021/18021 [==============================] - 1s 65us/step - loss: 1.6492e-04 - val_loss: 9.0968e-05
Epoch 13/24
18021/18021 [==============================] - 1s 65us/step - loss: 1.3988e-04 - val_loss: 7.3663e-05
Epoch 14/24
18021/18021 [==============================] - 1s 65us/step - loss: 1.2871e-04 - val_loss: 7.1783e-05
Epoch 15/24
18021/18021 [==============================] - 1s 65us/step - loss: 1.5244e-04 - val_loss: 7.7632e-05
Epoch 16/24
18021/18021 [==============================] - 1s 65us/step - loss: 1.2748e-04 - val_loss: 6.6736e-05
Epoch 17/24
18021/18021 [==============================] - 1s 69us/step - loss: 1.3658e-04 - val_loss: 1.1194e-04
Epoch 18/24
18021/18021 [==============================] - 1s 65us/step - loss: 1.7179e-04 - val_loss: 1.1059e-04
Epoch 19/24
18021/18021 [==============================] - 1s 65us/step - loss: 1.4699e-04 - val_loss: 6.8268e-05
Epoch 20/24
18021/18021 [==============================] - 1s 65us/step - loss: 1.2795e-04 - val_los

Epoch 12/24
18021/18021 [==============================] - 1s 68us/step - loss: 6.5086e-04 - val_loss: 6.1585e-04
Epoch 13/24
18021/18021 [==============================] - 1s 68us/step - loss: 6.4408e-04 - val_loss: 5.3921e-04
Epoch 14/24
18021/18021 [==============================] - 1s 68us/step - loss: 6.1731e-04 - val_loss: 3.6908e-04
Epoch 15/24
18021/18021 [==============================] - 1s 68us/step - loss: 6.1366e-04 - val_loss: 6.9905e-04
Epoch 16/24
18021/18021 [==============================] - 1s 68us/step - loss: 6.6649e-04 - val_loss: 4.2763e-04
Epoch 17/24
18021/18021 [==============================] - 1s 68us/step - loss: 4.4271e-04 - val_loss: 6.3463e-04
Epoch 18/24
18021/18021 [==============================] - 1s 68us/step - loss: 7.7545e-04 - val_loss: 3.9726e-04
Epoch 19/24
18021/18021 [==============================] - 1s 68us/step - loss: 4.0730e-04 - val_loss: 5.5087e-04
Epoch 20/24
18021/18021 [==============================] - 1s 68us/step - loss: 6.2358e-

In [12]:
best_models[0][1].history['val_loss']

[6.536736318244562e-05,
 6.510217520013694e-05,
 6.513846950387909e-05,
 6.518134240483755e-05,
 6.516029419099282e-05,
 6.508750844065597e-05,
 6.51531610335822e-05,
 6.528300888653537e-05,
 6.522818940623765e-05,
 6.507724339215208e-05,
 6.507940502176529e-05,
 6.508373916041907e-05,
 6.528326609327166e-05,
 6.517413978982057e-05,
 6.516693059447548e-05,
 6.526266017446494e-05,
 6.515097473182516e-05,
 6.514436119511892e-05,
 6.512688258133633e-05,
 6.51181755049757e-05,
 6.506902700157801e-05,
 6.515684092270431e-05,
 6.51403212296431e-05,
 6.512177953778983e-05]

In [13]:
print('optimal parameters')
print('_______________________')
for key,val in best_models[0][2].items():
    if key != 'x' and key != 'y': print(f'{key}: {val}')

optimal parameters
_______________________
lstmsize: 4
optimizer: adagrad
twice: True
dropout: 0.2
merge_density: 208
shuffle: True
full_density: True
activation: relu
density: 224


In [14]:
best_models[0][0].summary()

Model: "model_130"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lstm_258_input (InputLayer)     (None, 4, 99)        0                                            
__________________________________________________________________________________________________
lstm_258 (LSTM)                 (None, 4, 4)         1664        lstm_258_input[0][0]             
__________________________________________________________________________________________________
dropout_257 (Dropout)           (None, 4, 4)         0           lstm_258[0][0]                   
__________________________________________________________________________________________________
lstm_259 (LSTM)                 (None, 4)            144         dropout_257[0][0]                
__________________________________________________________________________________________

In [15]:
checkpoint = ModelCheckpoint(
    filepath='./checkpoints/fundamental_boosted.{epoch}-{val_loss}.hdf5',
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [16]:
params = best_models[0][2]
params['x_time'] = train_time
params['x_nexus'] = train_nexus
params['y'] = train_y
params['epochs'] = 2000
params['callbacks'] = [checkpoint]

In [18]:
final_model = build_complex(**params)

Train on 18021 samples, validate on 2003 samples
Epoch 1/2000
18021/18021 [==============================] - 44s 2ms/step - loss: 0.0083 - val_loss: 6.5165e-05
Epoch 2/2000
18021/18021 [==============================] - 1s 69us/step - loss: 1.2284e-04 - val_loss: 6.5804e-05
Epoch 3/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2275e-04 - val_loss: 6.5394e-05
Epoch 4/2000
18021/18021 [==============================] - 1s 70us/step - loss: 1.2264e-04 - val_loss: 6.5835e-05
Epoch 5/2000
18021/18021 [==============================] - 1s 71us/step - loss: 1.2275e-04 - val_loss: 6.5168e-05
Epoch 6/2000
18021/18021 [==============================] - 1s 69us/step - loss: 1.2285e-04 - val_loss: 6.5286e-05
Epoch 7/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2274e-04 - val_loss: 6.5218e-05
Epoch 8/2000
18021/18021 [==============================] - 1s 69us/step - loss: 1.2258e-04 - val_loss: 6.5221e-05
Epoch 9/2000
18021/18021 [=========

18021/18021 [==============================] - 1s 68us/step - loss: 1.2239e-04 - val_loss: 6.5193e-05
Epoch 72/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2253e-04 - val_loss: 6.5054e-05
Epoch 73/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2230e-04 - val_loss: 6.5059e-05
Epoch 74/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2245e-04 - val_loss: 6.5066e-05
Epoch 75/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2241e-04 - val_loss: 6.5232e-05
Epoch 76/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2246e-04 - val_loss: 6.5291e-05
Epoch 77/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2239e-04 - val_loss: 6.5106e-05
Epoch 78/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2247e-04 - val_loss: 6.5620e-05
Epoch 79/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.22

18021/18021 [==============================] - 1s 68us/step - loss: 1.2243e-04 - val_loss: 6.5314e-05
Epoch 142/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2241e-04 - val_loss: 6.5044e-05
Epoch 143/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2216e-04 - val_loss: 7.0662e-05
Epoch 144/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2263e-04 - val_loss: 6.5047e-05
Epoch 145/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2240e-04 - val_loss: 6.5074e-05
Epoch 146/2000
18021/18021 [==============================] - 1s 71us/step - loss: 1.2240e-04 - val_loss: 6.5134e-05
Epoch 147/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2243e-04 - val_loss: 6.5093e-05
Epoch 148/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2228e-04 - val_loss: 6.5078e-05
Epoch 149/2000
18021/18021 [==============================] - 1s 68us/step - lo

18021/18021 [==============================] - 1s 72us/step - loss: 1.2238e-04 - val_loss: 6.5034e-05
Epoch 212/2000
18021/18021 [==============================] - 1s 69us/step - loss: 1.2240e-04 - val_loss: 6.5204e-05
Epoch 213/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2237e-04 - val_loss: 6.5311e-05
Epoch 214/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2232e-04 - val_loss: 6.5244e-05
Epoch 215/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2245e-04 - val_loss: 6.5175e-05
Epoch 216/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2243e-04 - val_loss: 6.5068e-05
Epoch 217/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2237e-04 - val_loss: 6.5087e-05
Epoch 218/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2242e-04 - val_loss: 6.5266e-05
Epoch 219/2000
18021/18021 [==============================] - 1s 68us/step - lo

18021/18021 [==============================] - 1s 69us/step - loss: 1.2243e-04 - val_loss: 6.5049e-05
Epoch 282/2000
18021/18021 [==============================] - 1s 69us/step - loss: 1.2233e-04 - val_loss: 6.5159e-05
Epoch 283/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2229e-04 - val_loss: 6.5273e-05
Epoch 284/2000
18021/18021 [==============================] - 1s 69us/step - loss: 1.2232e-04 - val_loss: 6.5046e-05
Epoch 285/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2239e-04 - val_loss: 6.5235e-05
Epoch 286/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2234e-04 - val_loss: 6.5319e-05
Epoch 287/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2230e-04 - val_loss: 6.5046e-05
Epoch 288/2000
18021/18021 [==============================] - 1s 70us/step - loss: 1.2244e-04 - val_loss: 6.5047e-05
Epoch 289/2000
18021/18021 [==============================] - 1s 68us/step - lo

...

18021/18021 [==============================] - 1s 68us/step - loss: 1.2202e-04 - val_loss: 6.4999e-05
Epoch 1672/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2205e-04 - val_loss: 6.5006e-05
Epoch 1673/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2200e-04 - val_loss: 6.5082e-05
Epoch 1674/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2200e-04 - val_loss: 6.5116e-05
Epoch 1675/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2203e-04 - val_loss: 6.5221e-05
Epoch 1676/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2204e-04 - val_loss: 6.5155e-05
Epoch 1677/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2207e-04 - val_loss: 6.5230e-05
Epoch 1678/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2200e-04 - val_loss: 6.5056e-05
Epoch 1679/2000
18021/18021 [==============================] - 1s 67us/s

18021/18021 [==============================] - 1s 67us/step - loss: 1.2202e-04 - val_loss: 6.5208e-05
Epoch 1741/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2204e-04 - val_loss: 6.5066e-05
Epoch 1742/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2202e-04 - val_loss: 6.4990e-05
Epoch 1743/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2195e-04 - val_loss: 6.5234e-05
Epoch 1744/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2197e-04 - val_loss: 6.4990e-05
Epoch 1745/2000
18021/18021 [==============================] - 1s 70us/step - loss: 1.2201e-04 - val_loss: 6.4993e-05
Epoch 1746/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2203e-04 - val_loss: 6.5098e-05
Epoch 1747/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2200e-04 - val_loss: 6.5190e-05
Epoch 1748/2000
18021/18021 [==============================] - 1s 68us/s

18021/18021 [==============================] - 1s 69us/step - loss: 1.2194e-04 - val_loss: 6.4985e-05
Epoch 1810/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2198e-04 - val_loss: 6.5166e-05
Epoch 1811/2000
18021/18021 [==============================] - 1s 71us/step - loss: 1.2192e-04 - val_loss: 6.5263e-05
Epoch 1812/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2198e-04 - val_loss: 6.5066e-05
Epoch 1813/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2204e-04 - val_loss: 6.5405e-05
Epoch 1814/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2195e-04 - val_loss: 6.4984e-05
Epoch 1815/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2199e-04 - val_loss: 6.5516e-05
Epoch 1816/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2205e-04 - val_loss: 6.5167e-05
Epoch 1817/2000
18021/18021 [==============================] - 1s 67us/s

18021/18021 [==============================] - 1s 67us/step - loss: 1.2196e-04 - val_loss: 6.5049e-05
Epoch 1879/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2198e-04 - val_loss: 6.5052e-05
Epoch 1880/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2201e-04 - val_loss: 6.5072e-05
Epoch 1881/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2204e-04 - val_loss: 7.7838e-05
Epoch 1882/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2254e-04 - val_loss: 6.5038e-05
Epoch 1883/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2199e-04 - val_loss: 6.5041e-05
Epoch 1884/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2206e-04 - val_loss: 6.4991e-05
Epoch 1885/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2196e-04 - val_loss: 6.5210e-05
Epoch 1886/2000
18021/18021 [==============================] - 1s 67us/s

18021/18021 [==============================] - 1s 68us/step - loss: 1.2198e-04 - val_loss: 6.4958e-05
Epoch 1948/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2195e-04 - val_loss: 6.5044e-05
Epoch 1949/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2198e-04 - val_loss: 6.5241e-05
Epoch 1950/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2201e-04 - val_loss: 6.5058e-05
Epoch 1951/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2194e-04 - val_loss: 6.4962e-05
Epoch 1952/2000
18021/18021 [==============================] - 1s 68us/step - loss: 1.2195e-04 - val_loss: 6.4958e-05
Epoch 1953/2000
18021/18021 [==============================] - 1s 67us/step - loss: 1.2196e-04 - val_loss: 8.0988e-05
Epoch 1954/2000
18021/18021 [==============================] - 1s 69us/step - loss: 1.2265e-04 - val_loss: 6.5150e-05
Epoch 1955/2000
18021/18021 [==============================] - 1s 69us/s

In [19]:
final_model[2]

{'lstmsize': 4,
 'optimizer': 'adagrad',
 'twice': True,
 'dropout': 0.2,
 'merge_density': 208,
 'shuffle': True,
 'full_density': True,
 'activation': 'relu',
 'density': 224,
 'callbacks': [<keras.callbacks.callbacks.ModelCheckpoint at 0x1eab54e2408>]}

In [20]:
final_model[0].summary()

Model: "model_182"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lstm_362_input (InputLayer)     (None, 4, 99)        0                                            
__________________________________________________________________________________________________
lstm_362 (LSTM)                 (None, 4, 4)         1664        lstm_362_input[0][0]             
__________________________________________________________________________________________________
dropout_361 (Dropout)           (None, 4, 4)         0           lstm_362[0][0]                   
__________________________________________________________________________________________________
lstm_363 (LSTM)                 (None, 4)            144         dropout_361[0][0]                
__________________________________________________________________________________________

In [30]:
final_model[0].load_weights('./checkpoints/fundamental_boosted.(best).hdf5')
y_predicted = final_model[0].predict([test_time, test_nexus])

In [31]:
predicted_y_test = y_normaliser.inverse_transform(y_predicted)
true_y_test = y_normaliser.inverse_transform(test_y)

In [32]:
loss = np.mean((true_y_test - predicted_y_test)**2)
medium_error = np.mean(abs(true_y_test - predicted_y_test))
print(f'MSE is {loss:.2f}\nMedium error is {medium_error:.2f}')

MSE is 34444.53
Medium error is 40.37


In [33]:
print(f'Trend accuracy is: {(sum(1 for r,p in zip(predicted_y_test,true_y_test) if (r > 0 and p > 0) or (r < 0 and p < 0))/len(test_y))*100:.2f}%')

Trend accuracy is: 51.69%


In [34]:
print('Test growth trends:        Predicted growth trends:')
print(sum(1 for v in true_y_test if v > 0),'                     ',sum(1 for v in predicted_y_test if v > 0))
print('Test decreasing trends:    Predicted decreasing trends:')
print(sum(1 for v in true_y_test if v < 0),'                     ',sum(1 for v in predicted_y_test if v < 0))

Test growth trends:        Predicted growth trends:
1161                       2144
Test decreasing trends:    Predicted decreasing trends:
1064                       81


In [35]:
print(f'Test mean: {np.mean(true_y_test)}')
print(f'Predictions mean: {np.mean(predicted_y_test)}')

Test mean: 14.477038088235396
Predictions mean: 18.898700714111328


##### Results seem to be random around some tipical values

In [36]:
print('Test values:    Predicted values:')
for r,p in zip(true_y_test.flatten(),predicted_y_test.flatten()):
    print(f'    {r:.2f}    {p:>12.2f}')

Test values:    Predicted values:
    5.57            3.33
    6.17            4.23
    11.43            3.51
    10.04            3.23
    3.54            3.03
    -11.71            3.69
    -8.47            2.25
    32.34            3.46
    -24.28            1.87
    22.76            2.76
    11.66            2.65
    -13.97            2.56
    7.42            3.49
    -23.65            4.87
    35.69            3.16
    29.35            3.31
    -10.83            3.25
    1.16            3.17
    2.99            3.14
    -3.80            3.02
    -2.20            2.93
    4.72            2.61
    3.13            2.88
    -15.51            2.79
    18.29            1.99
    -1.13            1.47
    2.59            1.37
    1.17            1.54
    -32.48            1.27
    5.24            2.42
    -1.25           34.41
    17.59           34.08
    -4.33           34.03
    11.52           33.93
    5.17           33.91
    9.82           33.97
    20.77           33.85
    20.90 

    3.48            3.96
    24.46            3.70
    24.94            4.01
    4.72            6.15
    21.03            5.80
    9.85            8.45
    -38.35            8.40
    15.01            5.66
    12.85            7.86
    -0.26            7.95
    -34.42            7.68
    -2.43            5.25
    39.14            7.95
    -66.62           27.82
    66.45           20.35
    2.79           41.72
    -1.39           40.58
    -4.81           43.92
    -20.82           39.38
    8.57           43.26
    5.10           30.68
    -13.37           33.69
    3.51           35.27
    -60.26           25.40
    43.58           18.40
    30.17           32.16
    6.62           31.99
    11.18           32.11
    31.84           31.73
    -11.44           31.55
    36.60           31.36
    52.54           31.14
    5.63           30.35
    -17.85           29.74
    -17.91           28.72
    0.10           28.78
    -4.93           28.33
    58.27           28.14
    -4.67    

    13.96            4.01
    0.13            4.34
    5.73            4.10
    6.88            3.96
    -2.11            4.21
    -0.23            4.32
    -8.93            6.30
    14.61            4.39
    9.45            5.61
    2.04            5.38
    11.16            5.49
    -33.00            5.70
    46.37            6.05
    8.26           21.96
    -18.49           19.09
    12.00           23.41
    0.10           23.01
    23.28           20.91
    -26.93           24.35
    -0.98           26.15
    -6.61            8.03
    -21.58            7.80
    6.17            7.83
    -12.61            7.73
    -15.24            7.78
    -7.46            7.94
    -51.86            7.75
    48.93            8.05
    16.49           27.23
    -10.29           27.32
    -0.71           27.99
    1.79           28.28
    4.72           27.90
    13.06           27.58
    8.33           27.89
    -3.35           28.23
    -17.08           27.73
    6.14           26.72
    -13.73     